In [1]:
import pandas as pd
import numpy as np

from bs4 import BeautifulSoup
from scrapy.selector import Selector

import re, requests, csv
import lxml

# Regex
regex_url = r"(http(s?):\/\/(www)?[A-Za-z0-9-?\/\.]+)"
regex_hastags = r"(#[A-Za-z0-9]{2,})"

# File url
file_url = '../datasets/bbchealth.txt'

# Basic list
csv_header = ['tweetid','date','title']

# Support scripts
%run -i scripts.py

In [2]:
regex_bbc = r"(http(s?):\/\/(www)?[A-Za-z0-9\/\.]+)"
# regex_cbc = r"(http(s?):\/\/(www)?[A-Za-z0-9-?=\s\/\.]+)"
xpath_bbc = ['//div[@class="story-body__inner"]/p/text()',
            '//ul/li[@class="tags-list__tags"]/a/text()',
            '//span[@class="byline__name"]/text()']

# Regex "&amp"
regex_decode = r"(\&amp)"

In [3]:
# -------------------------------------------------
frame = pd.read_table(file_url, sep='|', header= None, names = csv_header, encoding="utf-8")
frame.head()

,tweetid,date,title
0,585978391360221184,Thu Apr 09 01:31:50 +0000 2015,Breast cancer risk test devised http://bbc.in/...
1,585947808772960257,Wed Apr 08 23:30:18 +0000 2015,GP workload harming care - BMA poll http://bbc...
2,585947807816650752,Wed Apr 08 23:30:18 +0000 2015,Short people's 'heart risk greater' http://bbc...
3,585866060991078401,Wed Apr 08 18:05:28 +0000 2015,New approach against HIV 'promising' http://bb...
4,585794106170839041,Wed Apr 08 13:19:33 +0000 2015,Coalition 'undermined NHS' - doctors http://bb...


In [4]:
frame.shape

(3929, 3)

In [5]:
# -------------------------------------------------
frame.loc[:,'title'][0]

'Breast cancer risk test devised http://bbc.in/1CimpJF'

In [6]:
# # Split Url out of content
# def get_url(regex, sample):
#     result = re.search(regex, sample)[0]
    
#     return result

# # Split text out of content
# def split_txt_form_url(regex, sample):
#     url = get_url(regex, sample)
#     result = sample.strip(url)
    
#     return result

# # Scrapt content base on xpath command.
# def scrapt_w_xpath(xpath_cmd):
#     result = dict()
    
#     # Content
#     contents = xpath_selector.xpath(xpath_cmd[0]).extract()
#     result.update({'contents': contents})
    
#     # Tags
#     tags = xpath_selector.xpath(xpath_cmd[1]).extract()
#     result.update({'tags': tags})
    
#     # Author
#     author = xpath_selector.xpath(xpath_cmd[2]).extract()
#     result.update({'author': author})
    
#     return result

In [7]:
# -------------------------------------------------
# Split Url & Text
frame['url'] = frame['title'].map(lambda row: get_url(regex_url, row))
frame['title'] = frame['title'].map(split_txt_form_url)
frame.head()

http://bbc.in/1CimpJF
http://bbc.in/1ChTBRv
http://bbc.in/1ChTANp
http://bbc.in/1E6jAjt
http://bbc.in/1CnLwK7
http://bbc.in/1Ffj6ci
http://bbc.in/1N7wSSz
http://bbc.in/1CmrRu3
http://bbc.in/1FdSGrl
http://bbc.in/1Ci5eqD
http://bbc.in/1Ci5c22
http://bbc.in/1Fa4Xgz
http://bbc.in/1CfcVOb
http://bbc.in/1Ceq0Y7
http://bbc.in/1C5Mlbk
http://bbc.in/1C9Gy3o
http://bbc.in/1F3nif2
http://bbc.in/1Pb4Xjb
http://bbc.in/1ETauYE
http://bbc.in/1CD3Dip
http://bbc.in/19RKqiY
http://bbc.in/1BTQU7k
http://bbc.in/1ENwsMD
http://bbc.in/1BT4s2U
http://bbc.in/1BSKo0T
http://bbc.in/1BREQDJ
http://bbc.in/1EKlFCK
http://bbc.in/1P1AJyX
http://bbc.in/1BO9jlZ
http://bbc.in/1P0rpv2
http://bbc.in/1BO3eWQ
http://bbc.in/1BRNoeJ
http://bbc.in/1MvP0oO
http://bbc.in/19Ad4Vy
http://bbc.in/1OWzSPT
http://bbc.in/1BIUAZx
http://bbc.in/1BLrny7
http://bbc.in/1BLlj8L
http://bbc.in/1BKT2iJ
http://bbc.in/1DhbCnf
http://bbc.in/1DhbCn6
http://bbc.in/1ysI93N
http://bbc.in/1G0J4Ag
http://bbc.in/1MjhH8r
http://bbc.in/1BGLzBf
http://bbc

http://bbc.in/1onsZIp
http://bbc.in/1ont1jl
http://bbc.in/1rx1XTU
http://bbc.in/1m0OH6Z
http://bbc.in/1j2Snp3
http://bbc.in/1oh1T5K
http://bbc.in/1s6FkDo
http://bbc.in/1mYL55D
http://bbc.in/1lYsOoQ
http://bbc.in/1j1RxZV
http://bbc.in/1j1RwVE
http://bbc.in/1ofpLXr
http://bbc.in/1kjNGSS
http://bbc.in/1oda9ng
http://bbc.in/1j0vPW2
http://bbc.in/1j0gvJ5
http://bbc.in/1mWkJB2
http://bbc.in/1lVmDSh
http://bbc.in/1mVo2IU
http://bbc.in/1vyyI0E
http://bbc.in/1vyyI0B
http://bbc.in/1rZVBKj
http://bbc.in/1vx1Xkm
http://bbc.in/1mUmVZP
http://bbc.in/1mkbFXL
http://bbc.in/1rX44hi
http://bbc.in/1o7v76M
http://bbc.in/1iXaUTI
http://bbc.in/1rhlNCz
http://bbc.in/1o7ltRI
http://bbc.in/1rW8KEa
http://bbc.in/1vue9CB
http://bbc.in/1iWz68G
http://bbc.in/1o5HIrk
http://bbc.in/1iVuY9d
http://bbc.in/1rcOXCI
http://bbc.in/1rbQg4S
http://bbc.in/1rb4oLM
http://bbc.in/1o25WCQ
http://bbc.in/1k94Cve
http://bbc.in/1mg4JuO
http://bbc.in/1lLYI7V
http://bbc.in/1rRGQcg
http://bbc.in/1o0VPxZ
http://bbc.in/1k8aVzk
http://bbc

http://bbc.in/1bpWnLB
http://bbc.in/1bpWmqZ
http://bbc.in/1bpWnvg
http://bbc.in/1aW8SNj
http://bbc.in/1boaalQ
http://bbc.in/IjfTg1
http://bbc.in/1eA9etp
http://bbc.in/1ifdeEL
http://bbc.in/1iSL9Rt
http://bbc.in/1iSixHO
http://bbc.in/ImQ3sf
http://bbc.in/ImQ3se
http://bbc.in/1cqJSw9
http://bbc.in/1cqrd3n
http://bbc.in/1cqrenZ
http://bbc.in/IkWeNm
http://bbc.in/1aQRfhC
http://bbc.in/1cpW6oB
http://bbc.in/IjZ8Cb
http://bbc.in/1cp2A7m
http://bbc.in/1coHKVK
http://bbc.in/1iN9VlR
http://bbc.in/IiHX40
http://bbc.in/1iL4BiY
http://bbc.in/1iKC397
http://bbc.in/1ckUkVL
http://bbc.in/1i84vUS
http://bbc.in/1cihsUR
http://bbc.in/1cihsUQ
http://bbc.in/1dpNFPd
http://bbc.in/1cbOcPQ
http://bbc.in/1cbOcPJ
http://bbc.in/1c9Z3JV
http://bbc.in/1bXAjX2
http://bbc.in/1c8Ieio
http://bbc.in/1aB7lMj
http://bbc.in/1fprpF4
http://bbc.in/1foQQ9Z
http://bbc.in/18Uu9CF
http://bbc.in/1foQOPi
http://bbc.in/1ayMrgN
http://bbc.in/1fn2ouj
http://bbc.in/17QsuxQ
http://bbc.in/1bTE9Au
http://bbc.in/1flqBkH
http://bbc.in/18



http://bbc.in/1ycccvh
NHS staff 'afraid to speak out' http://bbc.in/1IPWvFk


http://bbc.in/1IPWvFk
VIDEO: Pioneering drug treatment for cancer http://bbc.in/1ycccva


http://bbc.in/1ycccva
Obese workers have disability rights http://bbc.in/195MwM6


http://bbc.in/195MwM6
'Private' NHS hospital wants bailout http://bbc.in/1E3h9Lj


http://bbc.in/1E3h9Lj
Food fight: Row over fat advice http://bbc.in/1y6QqsQ


http://bbc.in/1y6QqsQ
VIDEO: Health benefits of oil explained http://bbc.in/1uzEtAB


http://bbc.in/1uzEtAB
Damages for France baby mix-up http://bbc.in/1uzEvbz


http://bbc.in/1uzEvbz
Advice: No alcohol safe in pregnancy http://bbc.in/178iUNm


http://bbc.in/178iUNm
Hospital maternity care concerns http://bbc.in/194pbdH


http://bbc.in/194pbdH
Putting fitness trackers to the test http://bbc.in/194pbdA


http://bbc.in/194pbdA
'Smart' insulin hope for diabetes http://bbc.in/1uyAl3W


http://bbc.in/1uyAl3W
'Focus on fitness not fatness' http://bbc.in/1y462xa


http://bbc.in/1y462xa

VIDEO: 'Shocking' disability access to shops http://bbc.in/1AArwoE


http://bbc.in/1AArwoE
Why 361 needles are better than one http://bbc.in/1IbQNe6


http://bbc.in/1IbQNe6
Shops' disability access 'shocking' http://bbc.in/1tU6XyY


http://bbc.in/1tU6XyY
Text reminders 'stop medicine waste' http://bbc.in/1AxAJhG


http://bbc.in/1AxAJhG
VIDEO: 'Why does my breastfeeding offend?' http://bbc.in/12If23t


http://bbc.in/12If23t
Ebola: 'basic care being missed' http://bbc.in/1267Htr


http://bbc.in/1267Htr
VIDEO: SWAN children given new hope http://bbc.in/1I9tL7G


http://bbc.in/1I9tL7G
Apology for wrong GP risk data http://bbc.in/1vvzAHk


http://bbc.in/1vvzAHk
VIDEO: 'We weren't told of kidney risks' http://bbc.in/1z1oCdv


http://bbc.in/1z1oCdv
Disabled man 'put up for auction' http://bbc.in/1tQj9B0


http://bbc.in/1tQj9B0
Breastfeed discreetly, urges Farage http://bbc.in/1ztHOiz


http://bbc.in/1ztHOiz
Nigerian medics join Ebola response http://bbc.in/1ztHOir


http://bbc.in/1ztHOir
Brai

...


http://bbc.in/1pTZW26
Big jump in Mers cases reported http://bbc.in/UMZtTN


http://bbc.in/UMZtTN
VIDEO: How anxiety found a voice http://bbc.in/1nLH5BJ


http://bbc.in/1nLH5BJ
VIDEO: Giving birth naturally in Brazil http://bbc.in/1jn5mN8


http://bbc.in/1jn5mN8
VIDEO: Can you trust your sunglasses? http://bbc.in/1jn9OLZ


http://bbc.in/1jn9OLZ
VIDEO: Abortion pill risk in Cambodia http://bbc.in/1jn9OLR


http://bbc.in/1jn9OLR
'Drastic action' needed on Ebola http://bbc.in/1rCzKbn


http://bbc.in/1rCzKbn
VIDEO: Hunt: 'We let Savile victims down' http://bbc.in/1jmjmqx


http://bbc.in/1jmjmqx
Man has 'balloon tumour' removed http://bbc.in/1yQPMmW


http://bbc.in/1yQPMmW
VIDEO: Aherne: 'Only special people get cancer' http://bbc.in/1lgcPxf


http://bbc.in/1lgcPxf
Call to halve target for added sugar http://bbc.in/1jm9xZD


http://bbc.in/1jm9xZD
Savile NHS victims aged five to 75 http://bbc.in/1yQvD0v


http://bbc.in/1yQvD0v
Savile hospital abuse reports due http://bbc.in/1rBrgS7


http:/



http://bbc.in/1hIL03y
Under 16s 'shut out of safe havens' http://bbc.in/1t8lI3q


http://bbc.in/1t8lI3q
Warning over bowel cancer screening http://bbc.in/1kw15Kp


http://bbc.in/1kw15Kp
VIDEO: Lord Tebbit turns children's writer http://bbc.in/1iWdN0O


http://bbc.in/1iWdN0O
'Terror' of patient's op wake-up http://bbc.in/1iUCL0B


http://bbc.in/1iUCL0B
Crohn's patient loses fertility case http://bbc.in/1eu5pKa


http://bbc.in/1eu5pKa
'Care failings' over man's op death http://bbc.in/1iTIi7v


http://bbc.in/1iTIi7v
AUDIO: RNIB: UK needs sight loss advisors http://bbc.in/Q95s38


http://bbc.in/Q95s38
GP out-of-hours care 'diabolical' http://bbc.in/1iRhHba


http://bbc.in/1iRhHba
VIDEO: Inside the 'celeb' rehab centre http://bbc.in/Q8EYPh


http://bbc.in/Q8EYPh
VIDEO: Sensors to prevent pain for amputees http://bbc.in/1iQihGr


http://bbc.in/1iQihGr
Sensors to prevent pain for amputees http://bbc.in/1hDiSif


http://bbc.in/1hDiSif
GP hours scheme 'to benefit 7m' http://bbc.in/1hzDFn0


h

AUDIO: Ex-drug user: addiction never leaves http://bbc.in/1aje3Jp


http://bbc.in/1aje3Jp
AUDIO: Drinking game 'form of bullying' http://bbc.in/1ajdsYa


http://bbc.in/1ajdsYa
VIDEO: Plastic surgery 'boom' in the UK http://bbc.in/1fRJoRp


http://bbc.in/1fRJoRp
Elderly breast cancer warning http://bbc.in/1fR4G1r


http://bbc.in/1fR4G1r
VIDEO: Illegally tattooed at the age of 13 http://bbc.in/1fQRfhX


http://bbc.in/1fQRfhX
Vitamin pills 'lower exercise gains' http://bbc.in/LFgH0Z


http://bbc.in/LFgH0Z
'Tinnitus risk' of noisy nights out http://bbc.in/1kAUq0h


http://bbc.in/1kAUq0h
'Pocket optician' trialled in schools http://bbc.in/1kAUnBM


http://bbc.in/1kAUnBM
Plastic surgery 'booming' in the UK http://bbc.in/LFgElP


http://bbc.in/LFgElP
Can couples really get stuck together during sex? http://bbc.in/LFgElK


http://bbc.in/LFgElK
Health watchdog attacks NHS culture http://bbc.in/1fMsrHW


http://bbc.in/1fMsrHW
VIDEO: NHS warned it 'could go bust' http://bbc.in/1i77sRy


http://bb

http://bbc.in/1cotiNL
Surgeon wins award for 95p invention http://bbc.in/16NjlLg


http://bbc.in/16NjlLg
Poultry markets 'huge bird flu role' http://bbc.in/19cwKv8


http://bbc.in/19cwKv8
HIV antibodies 'have potent impact' http://bbc.in/1aFa7hv


http://bbc.in/1aFa7hv
Warning on 'too many' A&amp;E admissions http://bbc.in/19cwMDm


http://bbc.in/19cwMDm
Internet chat 'has a positive side' http://bbc.in/1aFa4Cn


http://bbc.in/1aFa4Cn
New York to raise cigarette sale age http://bbc.in/1aJd6Y8


http://bbc.in/1aJd6Y8
Bat virus clues to origins of Sars http://bbc.in/1aIxYPh


http://bbc.in/1aIxYPh
Two London A&amp;E units downgraded http://bbc.in/1clwiKT


http://bbc.in/1clwiKT
Apology for Obamacare woes http://bbc.in/1aCZPyg


http://bbc.in/1aCZPyg
Neets are 'public health time bomb' http://bbc.in/1aCZMmj


http://bbc.in/1aCZMmj
Doctors 'right' to stop patient care http://bbc.in/19aG8zw


http://bbc.in/19aG8zw
'Fewer smokers' attempt to quit http://bbc.in/1ckYE7U


http://bbc.in/1ckYE7U

,tweetid,date,title,url
0,585978391360221184,Thu Apr 09 01:31:50 +0000 2015,Breast cancer risk test devised,http://bbc.in/1CimpJF
1,585947808772960257,Wed Apr 08 23:30:18 +0000 2015,GP workload harming care - BMA poll,http://bbc.in/1ChTBRv
2,585947807816650752,Wed Apr 08 23:30:18 +0000 2015,Short people's 'heart risk greater',http://bbc.in/1ChTANp
3,585866060991078401,Wed Apr 08 18:05:28 +0000 2015,New approach against HIV 'promising',http://bbc.in/1E6jAjt
4,585794106170839041,Wed Apr 08 13:19:33 +0000 2015,Coalition 'undermined NHS' - doctors,http://bbc.in/1CnLwK7


In [8]:
# -------------------------------------------------
# Scrapt form web
df_temp = pd.DataFrame()
frame_urls = frame['url'].values

In [9]:
# -------------------------------------------------
# Run for this jupyter notebook
# title_attempt = True
# for key, url in enumerate(frame_urls):
#     try:
#         # Start the connection
#         res = requests.get(url)        

#         # Check return header
#         if res.status_code == 200:
#             print("Header:  ", 200)
#             print(url)
            
#             # Initialize XPath
#             xpath_selector = Selector(text=res.text)
            
#             list_content = dict()
            
#             # Scrapt Text with XPath
#             contents = xpath_selector.xpath(xpath_bbc[0]).extract()
#             contents = join_txt(contents)
#             print(len(contents))
#             # Tags
#             tags = xpath_selector.xpath(xpath_bbc[1]).extract()
#             if len(tags):
#                 tags = tags
#             else:
#                 tags = ''
#             print(len(tags))
#             # Author
#             author = xpath_selector.xpath(xpath_bbc[2]).extract()
#             if len(author):
#                 author = author[0]
#             else:
#                 author = ''
#             print(len(author))
#             list_content.update({'contents': contents,'tags': tags,'author': author})
            
#             if title_attempt:
#                 print("TItle: ", title_attempt)
#                 df_temp = pd.DataFrame(columns = list(list_content.keys()))
#                 frame = pd.concat([frame, df_temp])
#                 print(frame.columns)                
#                 title_attempt = False
                
#             for k_, v_ in list_content.items():
#                 print(k_," : " ,len(v_))
#                 frame[k_].iloc[key] = v_
#             print("=====================\n")

#         else:
#             print("['ERRORS'] {} , at {}".format(res.status_code, url))
#     except:
#         pass
    

In [ ]:
is_attempt = True
for key, url in enumerate(frame_urls):
    try:
        # Start the connection
        res = requests.get(url)

        # Get return content as text
        text_html = res.text

        # Check return header
        if res.status_code == 200:
            print("Header:  ", 200)
            print(url)
    #         soup = BeautifulSoup(res.content, 'lxml')
            xpath_selector = Selector(text=text_html)

            # Scrapt content with XPath
            list_content = dict()            
            # Scrapt Text with XPath
            contents = xpath_selector.xpath(xpath_bbc[0]).extract()
            contents = join_txt(contents)
            print(len(contents))
            # Tags
            tags = xpath_selector.xpath(xpath_bbc[1]).extract()
            if len(tags):
                tags = tags
            else:
                tags = ''
            print(len(tags))
            # Author
            author = xpath_selector.xpath(xpath_bbc[2]).extract()
            if len(author):
                author = author[0]
            else:
                author = ''
            print(len(author))
            list_content.update({'contents': contents,'tags': tags,'author': author})

            # Add new columns
            if is_attempt:
                df_temp = pd.DataFrame(columns = list(list_content.keys()))
                frame = pd.concat([frame, df_temp])
                print(frame.columns)
                is_attempt = False

            # Add to dataFrame
            for k_, v_ in list_content.items():
                print(k_," : " ,len(v_))
                frame[k_].iloc[key] = v_
            print("==================\n")

        else:
            print("['ERRORS'] {} , at {}".format(res.status_code, url))
    except:
        pass

Header:   200
http://bbc.in/1CimpJF
3407
3
18
Index(['author', 'contents', 'date', 'tags', 'title', 'tweetid', 'url'], dtype='object')
contents  :  3407
tags  :  3
author  :  18



scripts.py:42: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  
C:\Users\Shaynie\Anaconda3\lib\site-packages\pandas\core\indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


Header:   200
http://bbc.in/1ChTBRv
3665
2
15
contents  :  3665
tags  :  2
author  :  15

Header:   200
http://bbc.in/1ChTANp
2659
3
18
contents  :  2659
tags  :  3
author  :  18

Header:   200
http://bbc.in/1E6jAjt
2659
2
18
contents  :  2659
tags  :  2
author  :  18

Header:   200
http://bbc.in/1CnLwK7
2066
2
0
contents  :  2066
tags  :  2
author  :  0

Header:   200
http://bbc.in/1Ffj6ci
1697
1
0
contents  :  1697
tags  :  1
author  :  0

Header:   200
http://bbc.in/1N7wSSz
0
0
0
contents  :  0
tags  :  0
author  :  0

Header:   200
http://bbc.in/1CmrRu3
0
0
0
contents  :  0
tags  :  0
author  :  0

Header:   200
http://bbc.in/1FdSGrl
4100
1
19
contents  :  4100
tags  :  1
author  :  19

Header:   200
http://bbc.in/1Ci5eqD
0
0
0
contents  :  0
tags  :  0
author  :  0

Header:   200
http://bbc.in/1Ci5c22
3180
1
15
contents  :  3180
tags  :  1
author  :  15

Header:   200
http://bbc.in/1Fa4Xgz
2841
2
15
contents  :  2841
tags  :  2
author  :  15

Header:   200
http://bbc.in/1CfcVOb
46

Header:   200
http://bbc.in/1DS7Gua
0
0
0
contents  :  0
tags  :  0
author  :  0

Header:   200
http://bbc.in/1Gb99vn
4269
2
0
contents  :  4269
tags  :  2
author  :  0

Header:   200
http://bbc.in/1HuFsn2
0
0
0
contents  :  0
tags  :  0
author  :  0

Header:   200
http://bbc.in/1HtW1zE
8150
2
15
contents  :  8150
tags  :  2
author  :  15

Header:   200
http://bbc.in/1HtVZb1
2072
2
0
contents  :  2072
tags  :  2
author  :  0

Header:   200
http://bbc.in/1DPcgcE
0
0
0
contents  :  0
tags  :  0
author  :  0

Header:   200
http://bbc.in/1GLFMOk
3627
1
19
contents  :  3627
tags  :  1
author  :  19

Header:   200
http://bbc.in/1G4Oxoy
2086
3
0
contents  :  2086
tags  :  3
author  :  0

Header:   200
http://bbc.in/1DO1u6w
0
0
0
contents  :  0
tags  :  0
author  :  0

Header:   200
http://bbc.in/1G3XMW6
4424
1
0
contents  :  4424
tags  :  1
author  :  0

Header:   200
http://bbc.in/1LMYlYT
0
0
0
contents  :  0
tags  :  0
author  :  0

Header:   200
http://bbc.in/1GJlBAo
4240
3
18
contents  : 

Header:   200
http://bbc.in/1Fb1BIA
0
0
0
contents  :  0
tags  :  0
author  :  0

Header:   200
http://bbc.in/1D4YTEO
0
0
0
contents  :  0
tags  :  0
author  :  0

Header:   200
http://bbc.in/1Hy9aZ1
2342
1
15
contents  :  2342
tags  :  1
author  :  15

Header:   200
http://bbc.in/1Gpm5LQ
0
0
0
contents  :  0
tags  :  0
author  :  0

Header:   200
http://bbc.in/1Huc6WD
3010
3
15
contents  :  3010
tags  :  3
author  :  15

Header:   200
http://bbc.in/1F6yxC3
0
0
0
contents  :  0
tags  :  0
author  :  0

Header:   200
http://bbc.in/1CXFzcw
3374
1
19
contents  :  3374
tags  :  1
author  :  19

Header:   200
http://bbc.in/1HnG66y
3032
1
18
contents  :  3032
tags  :  1
author  :  18

Header:   200
http://bbc.in/1EJEbt9
2350
2
10
contents  :  2350
tags  :  2
author  :  10

Header:   200
http://bbc.in/1HjY0qC
2330
2
18
contents  :  2330
tags  :  2
author  :  18

Header:   200
http://bbc.in/1HjY0al
2907
1
0
contents  :  2907
tags  :  1
author  :  0

Header:   200
http://bbc.in/1COjVHM
3504
3
1

In [ ]:
# test_url = 'http://bbc.in/14GifAt'
# res = requests.get(test_url)
# author = soup.find('span',{'class':'byline__name'}).text
# author

In [ ]:
bbc.head()

In [ ]:
# -------------------------------------------------
frame.head()

In [ ]:
# -------------------------------------------------
frame.shape

In [ ]:
frame = pd.concat([frame,df_temp],axis=1)
df_temp.head()

In [ ]:
frame.to_csv('./run_data/bbc_scrapt_v2.csv', sep='|', encoding='utf-8', index=False)

In [ ]:
# %run -i scripts.py
# bbc['Contents'] = bbc['Contents'].map(lambda row: join_txt(row))